# Introduction
In this document, we go step by step in the procedure of generating the terrestrial parameters at any available resolution for a particular region. 

Let us start introducing some terminology:

## Glossary
- USGS    : [The United States Geological Survey](https://www.usgs.gov/)
- SAGA    : It is an open-source software designed as a robust geospatioal anlysis package
- GIS     : Geographical Information System
- DEM     : Digital Elevation Model is a 3D CG representation of a terrain's surface
- CSV file: A CSV is a comma-separated values file, which allows data to be saved in a tabular format.
- Raster  : A raster is a file where is the data is stored as a matrix of cells also called pixeles. There exists a variaty of format of rasters, but in this document, we will work with three of them

    - **Rasters.img**: Usually, USGS save their raster in this format,
    - **Raster.tif** : It is the most used format for many applications
    - **Raster.sdat**: This is used by the package SAGA, which is required to the generate the terrestrail parameters

- Decimal Latitude and Longitude system: This is a way to express geographic coordinates in decimal degrees. It’s just another way to represent that same location in a different format. For example, Chimney Tops is located at the coordinates (35.631167, -83.481167), which means Latitude =  35.631167  and  Longitude = -83.481167 
    

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Defining the folder where the csv file are going to be stored
%run code/tools.py

## We define some constants here. 
- **pi** is the value of the constant pi
- **Rearth** is the radius of earth
- **dx** is the distance from the center to any of the sides to the desired region
- **Year** corresponds to the year wwhen we want to estimate the soil moisture
- **Month** corresponds to the month of the year we want estimate the soil moisture

In [ ]:
# Define constants
pi = math.pi
REarth = 6378137 # Earth radius
dx     = 600     # Distance from  from the center

Folder = f'{Path}/CSF_Files/'

In [ ]:
Folder

## Setting the name of the files

# 1. Selecting the Desired Region
In this example, we work on a region containing the coordinates of the the ignition fire of the Chimney tops region. 

In [ ]:
Lat  =  35.631167
Long = -83.481167

Check in google earth to very the [location](https://earth.google.com/web/@35.63292863,-83.49585774,960.7674918a,19382.48366525d,35y,142.04295599h,45.11788942t,0r/data=CjkaNxIxCgAZP6iLFMrQQUAhhjqscMveVMAqGzM1wrAzNyc1Mi4yIk4gODPCsDI4JzUyLjIiVxgCIAE). Define the 1 Km square region bordering that point. In this point we need to add and substract 500 meters to the lattitude and longitude.

![Drag Racing](Images/ChimeneyTops.png)

In [ ]:
Max_Lat   = Lat  + (dx / REarth) * (180 / pi)
Min_Lat   = Lat  - (dx / REarth) * (180 / pi)
Max_Lon   = Long + (dx / REarth) * (180 / pi) / math.cos(Lat * pi/180)
Min_lon   = Long - (dx / REarth) * (180 / pi) / math.cos(Lat * pi/180)

print("Max Lat (Ymax):", Max_Lat)
print("Min Lat (Ymin):", Min_Lat)
print("Max Lon (Xmax):", Max_Lon)
print("Min Lon (Xmin):", Min_lon)

# 2. Select Digital Elevation Model (DEM) from USGS

Visit the webpage of The National Map viewer and download platform  https://viewer.nationalmap.gov/basic/
### Click on "Enter Coords" in the top left panel
![Drag Racing](data/Images/Enter_Coord.png)

### Insert the coordinates in the following windows and click on “Add to Map”
![Drag Racing](data/Images/coordinates.png)

## In the left panel, select:
### Elevation Products (3DEP)
### The desired resolution, for this example, 1 meter DEM
###  Search Products
![Drag Racing](data/Images/search.png)

You do not need all of them, click on “Thumbnail”, and select just the products that cover your region of interest, and then click “view cart”. You will see that some tiles are spatially duplicated, as they are in areas overlaying two states, the same tiles are stored individually in different states’ topographic datasets. Select only one of the overlaying tiles.

![Drag Racing](data/Images/Products.png)

###  You can download the files one at the time, but it is recommended to Export URLs to text files and, then download them with a script

# 3. Download DEM files from USGS and Unzip them
Insert the name of the file where are the names of the tiles to be downloaded


In [ ]:
os.chdir(Path)
# Make a folder to store the DEM file
raster_folder = f'{Path}/data/Rasters'
isdir = os.path.isdir(raster_folder)

if not isdir:
    os.makedirs(raster_folder)

os.chdir(raster_folder)

#Location and name of the file containing the files to be downloaded
filename = f"{Path}/data/download/data.txt" 
# Download the files

download_dem(filename)    # Download the files
         
# Unzip the zip files


!unzip *.zip



# Create a list of DEM Files
List_of_Rasters = !ls *.tif


In [ ]:
List_of_Rasters

# 4. Merge all the DEM Rasters in a Unique Raster (Mosaicking)

We have seen that sometimes to cover a region we need many tiles. We need to merge all of these tiles in only one
![Drag Racing](data/Images/Mosaicking.png)

In [ ]:
mergin_raster(List_of_Rasters)
#!mv Mosaicking.img data/Rasters/

# 5. Cropping the Desired Region
The Mosaic i.e., the merged raster covers more than the desired region, so we need to crop such region from the mosaic.
![Drag Racing](data/Images/Cropping.png)

In [ ]:
Max_lat   =  3946476
Min_lat   =  3945276 
Max_lon   =   276019
Min_lon   =   274819


croping_region('Mosaicking.img', str(Min_lon), str(Max_lon), str(Min_lat), str(Max_lat), 'Cropped.tif')

# 6. Change the Format of the Desired Region to SDAT format
The terrestrial parameters are generated using a package of SAGA, which requires all the rasters are in **SDAT format**


In [ ]:
File_name_of_region = f'Cropped.tif'
convert_to_sdat(File_name_of_region)

# 7. Create the Terrestrial Parameters

The name of the file has to be input with **no extension**

In [ ]:
file = 'Cropped'
terrestrial_parameters(file)

# 8. Create a Stack with the Elevation and all the Terrestrial Parameters
 Four files are created  in this part:
- **A stack raster with all parameters in Latitude-Longitude format**
- ** A raster with all parameters in UTMformat **
- ** A CSV file with all parameters in Latitude-Longitude format **
- ** A CSV file with all parameters in UTM format **

They all are saved in the folder Rasters

In [ ]:
list_of_files = !ls *.sdat
command = [f'{Path}/code/CreatingStack.R'] + list_of_files
creating_stack(command)

In [ ]:
TiempoModelo['Model']

# 12. Change the Coordinates to UTM (Optional)
All the files are expressed in Latitude-Longitude format. We need to change soil moisture and elevation file to UTM format (meters)

In [ ]:
UTM_Files = []
for file in LatLongFiles:
    Input_file  =  file+'_latlong.csv'
    Output_file =  file+'_UTM.csv'
    UTM_Files = UTM_Files + [Output_file]
    
    # Changing the format
    changing_to_utm(Input_file, Output_file)